<p style = "font-size:40px; 
font-family: Helvetica; 
font-weight : bold; 
text-align: left; 
padding: 0px 15px; 
border-radius:3px">
	Sample Dataset
</p>

In [1]:
DATA_PATH = './hmData'

In [2]:
import pandas as pd
import os.path as osp

In [3]:
customers = pd.read_csv(osp.join(DATA_PATH, 'customers.csv'))
articles = pd.read_csv(osp.join(DATA_PATH, 'articles.csv'))
sample_submission = pd.read_csv(osp.join(DATA_PATH, 'sample_submission.csv'))

In [4]:
train = pd.read_csv(osp.join(DATA_PATH, 'transactions_train.csv'))
train.t_dat = pd.to_datetime(train.t_dat)

print(train.shape)
train.head()

(31788324, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [5]:
train['t_dat'] = pd.to_datetime(train['t_dat'], format="%Y-%m-%d")
train['month'] = train['t_dat'].dt.strftime('%m')
train['year'] = train['t_dat'].dt.strftime('%Y')
train.head()

,t_dat,customer_id,article_id,price,sales_channel_id,month,year
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,09,2018
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,09,2018
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,09,2018
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,09,2018
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,09,2018



	Those who purchased less than 5 articles


In [7]:
customer_count = train.groupby('customer_id', as_index=False)[['price']].count().sort_index()
not_cold_users = customer_count[customer_count.price > 5].index
customer_count[customer_count.price > 5]

,customer_id,price
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,21
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,86
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,18
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,13
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,6
...,...,...
1362273,ffff8f9ecdce722b5bab97fff68a6d1866492209bfe524...,9
1362276,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,51
1362277,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,84
1362278,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,45



	A person whose last purchase was three months ago


In [37]:
import datetime

customer_last_purchase = train[train.customer_id.isin(not_cold_users)].groupby('customer_id', as_index=False)[['t_dat']].last()
cold_inactive_users = customer_last_purchase[customer_last_purchase.t_dat < "2020-06-01"].index
cold_active_users = customer_last_purchase[customer_last_purchase.t_dat > "2020-06-01"].index
customer_last_purchase[customer_last_purchase.t_dat < "2020-06-01"]

,customer_id,t_dat


We have a three types of user.
1. **not_cold_users**: Those who purchased within 3 months and have purchased a total of 5 or more so far. 
2. **cold_inactive_users**: Those who have purchased less than 5 items and have not purchased in the last 3 months
3. **cold_active_users**: Those who have purchased less than 5 and have purchased in the last 3 months

### Because the characteristics of the three types of users are completely different, the data set is created according to the user ratio.

In [38]:
import random
import pandas as pd
from typing import Tuple

def generate_dataset(
    rate # dataset size rate
):
    new_not_cold_users = random.sample(not_cold_users.to_list(), round(len(not_cold_users) * rate))
    new_cold_inactive_users = random.sample(cold_inactive_users.to_list(), round(len(cold_inactive_users) * rate))
    new_cold_active_users = random.sample(cold_active_users.to_list(), round(len(cold_active_users) * rate))
    
    new_users = new_not_cold_users + new_cold_inactive_users + new_cold_active_users

    new_train = train[train.customer_id.isin(new_users)]
    new_customer = customers[customers.customer_id.isin(new_users)]

    return new_train, new_customer

In [39]:
tiny_train, tiny_customer = generate_dataset(0.001)


Empty DataFrame
Columns: [t_dat, customer_id, article_id, price, sales_channel_id, month, year]
Index: []


In [12]:
tiny_train.to_csv('data-sample/transactions_train.csv', index=False)
tiny_customer.to_csv('data-sample/customers.csv', index=False)
articles.to_csv('data-sample/articles.csv', index=False)
sample_submission.to_csv('data-sample/sample_submission.csv', index=False)
